In [21]:
import sys, os; sys.path.append('..')
import random
import pyzx as zx
import logging

In [22]:
log = logging.getLogger("stdout")
while log.hasHandlers(): log.removeHandler(log.handlers[0])
log.addHandler(logging.StreamHandler(sys.stdout))
log.setLevel(logging.INFO)

def basic_opt(c):
    c = c.split_phase_gates().to_basic_gates()
    return zx.optimize.basic_optimization(c).to_basic_gates()

In [23]:
c = zx.Circuit.load('../circuits/Fast/barenco_tof_4_before').to_basic_gates()
zx.draw(c)
print(c.stats())

Circuit barenco_tof_4_before on 7 qubits with 114 gates.
        56 is the T-count
        58 Cliffords among which 
        48 2-qubit gates (48 CNOT, 0 other) and
        10 Hadamard gates.


In [24]:
g = c.to_graph()
zx.full_reduce(g)
g.normalize()
zx.draw(g)

In [25]:
c1 = zx.extract_circuit(g.copy())
zx.draw(c1)

In [26]:
print('ORIGINAL:\n  ' + c.stats())
print('\nFR:\n  ' + c1.stats())

ORIGINAL:
  Circuit barenco_tof_4_before on 7 qubits with 114 gates.
        56 is the T-count
        58 Cliffords among which 
        48 2-qubit gates (48 CNOT, 0 other) and
        10 Hadamard gates.

FR:
  Circuit  on 7 qubits with 156 gates.
        28 is the T-count
        128 Cliffords among which 
        59 2-qubit gates (27 CNOT, 32 other) and
        68 Hadamard gates.


In [27]:
gt = c.to_graph()
gt = zx.simplify.teleport_reduce(gt)
ct = zx.Circuit.from_graph(gt)
ct = basic_opt(ct)
print('TR:\n  ' + ct.stats())

TR:
  Circuit  on 7 qubits with 95 gates.
        28 is the T-count
        67 Cliffords among which 
        44 2-qubit gates (38 CNOT, 6 other) and
        10 Hadamard gates.


In [28]:
random.seed(1337)
print(zx.sparsify.size(g))
g2 = zx.sparsify.pivot_anneal(g, iters=500, logger=log)
print(zx.sparsify.size(g2))

96
96 at iteration 1 ^
100 at iteration 8 ^
102 at iteration 9 ^
105 at iteration 10 ^
101 at iteration 11 v
101 at iteration 14 ^
95 at iteration 15 v
93 at iteration 18 v
96 at iteration 23 ^
100 at iteration 24 ^
103 at iteration 26 ^
102 at iteration 28 v
113 at iteration 29 ^
114 at iteration 30 ^
112 at iteration 31 v
99 at iteration 32 v
99 at iteration 35 ^
99 at iteration 36 ^
99 at iteration 37 ^
95 at iteration 38 v
94 at iteration 39 v
92 at iteration 41 v
89 at iteration 43 v
90 at iteration 44 ^
91 at iteration 46 ^
99 at iteration 47 ^
101 at iteration 50 ^
98 at iteration 52 v
97 at iteration 54 v
96 at iteration 55 v
96 at iteration 56 ^
106 at iteration 57 ^
104 at iteration 60 v
97 at iteration 62 v
98 at iteration 65 ^
96 at iteration 68 v
99 at iteration 72 ^
94 at iteration 73 v
95 at iteration 74 ^
95 at iteration 76 ^
95 at iteration 77 ^
95 at iteration 78 ^
97 at iteration 79 ^
95 at iteration 81 v
95 at iteration 82 ^
97 at iteration 83 ^
98 at iteration 85 ^

In [29]:
c2 = zx.extract_circuit(g2.copy())
print('ORIGINAL:\n  ' + c.stats())
print('\nTR:\n  ' + ct.stats())
print('\nFR:\n  ' + c1.stats())
print('\nFR + SPARSE:\n  ' + c2.stats())

ORIGINAL:
  Circuit barenco_tof_4_before on 7 qubits with 114 gates.
        56 is the T-count
        58 Cliffords among which 
        48 2-qubit gates (48 CNOT, 0 other) and
        10 Hadamard gates.

TR:
  Circuit  on 7 qubits with 95 gates.
        28 is the T-count
        67 Cliffords among which 
        44 2-qubit gates (38 CNOT, 6 other) and
        10 Hadamard gates.

FR:
  Circuit  on 7 qubits with 156 gates.
        28 is the T-count
        128 Cliffords among which 
        59 2-qubit gates (27 CNOT, 32 other) and
        68 Hadamard gates.

FR + SPARSE:
  Circuit  on 7 qubits with 143 gates.
        25 is the T-count
        118 Cliffords among which 
        51 2-qubit gates (23 CNOT, 28 other) and
        66 Hadamard gates.


In [30]:
def twoq_score(g):
    c = basic_opt(zx.extract_circuit(g.copy()))
    return c.twoqubitcount()

In [31]:
random.seed(1337)
print(twoq_score(g))
g3 = zx.sparsify.pivot_anneal(g, iters=500, cool=0.01, score=twoq_score, logger=log)
print(twoq_score(g3))

48
41 at iteration 2 v
44 at iteration 5 ^
51 at iteration 6 ^
55 at iteration 7 ^
52 at iteration 8 v
48 at iteration 10 v
46 at iteration 13 v
49 at iteration 16 ^
42 at iteration 18 v
43 at iteration 19 ^
39 at iteration 24 v
44 at iteration 35 ^
53 at iteration 36 ^
48 at iteration 37 v
52 at iteration 45 ^
54 at iteration 47 ^
59 at iteration 48 ^
58 at iteration 49 v
54 at iteration 50 v
52 at iteration 51 v
49 at iteration 52 v
51 at iteration 54 ^
54 at iteration 55 ^
54 at iteration 56 ^
53 at iteration 58 v
56 at iteration 60 ^
52 at iteration 61 v
52 at iteration 62 ^
53 at iteration 63 ^
52 at iteration 64 v
51 at iteration 65 v
47 at iteration 67 v
48 at iteration 68 ^
53 at iteration 70 ^
53 at iteration 71 ^
52 at iteration 72 v
51 at iteration 73 v
46 at iteration 74 v
45 at iteration 75 v
43 at iteration 77 v
42 at iteration 79 v
42 at iteration 87 ^
46 at iteration 88 ^
41 at iteration 90 v
44 at iteration 92 ^
37 at iteration 94 v
35 at iteration 103 v
38 at iteratio

In [32]:
c3 = zx.extract_circuit(g3.copy())
print('ORIGINAL:\n  ' + c.stats())
print('\nTR:\n  ' + ct.stats())
print('\nFR:\n  ' + c1.stats())
print('\nFR + SPARSE*:\n  ' + c3.stats())

ORIGINAL:
  Circuit barenco_tof_4_before on 7 qubits with 114 gates.
        56 is the T-count
        58 Cliffords among which 
        48 2-qubit gates (48 CNOT, 0 other) and
        10 Hadamard gates.

TR:
  Circuit  on 7 qubits with 95 gates.
        28 is the T-count
        67 Cliffords among which 
        44 2-qubit gates (38 CNOT, 6 other) and
        10 Hadamard gates.

FR:
  Circuit  on 7 qubits with 156 gates.
        28 is the T-count
        128 Cliffords among which 
        59 2-qubit gates (27 CNOT, 32 other) and
        68 Hadamard gates.

FR + SPARSE*:
  Circuit  on 7 qubits with 145 gates.
        26 is the T-count
        119 Cliffords among which 
        50 2-qubit gates (16 CNOT, 34 other) and
        62 Hadamard gates.
